<a href="https://colab.research.google.com/github/acts701/LLM/blob/main/241230_%EA%B0%9C%EB%B0%9C%EC%9D%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비 상태 요약
1. MP3 데이터를 텍스트로 변환:
    - OpenAI Whisper를 사용해 음성 파일을 텍스트로 변환.
    - 약 60분 분량의 음성 데이터를 텍스트로 확보.
        - 이 과정에서 근원적(?) 문제가 생긴다.         
        - 유료 버전을 쓰기에 부담되고 무료 버전은 품질 문제가 있음        
2. 문장 단위로 분리
    - KSS(Korean Sentence Splitter) 라이브러리를 사용해 텍스트 데이터를 문장 단위로 나눔.
3. Embedding Vector 생성
    - 문장 데이터를 벡터화하여 JSON 형태로 저장.
4. NLI 데이터셋 생성:
    - 모든 텍스트 파일에서 NLI 데이터셋 50개를 생성.
    - 이것도 chat gpt api로 자동으로 만들었는데 하나씩 열어서 보면 품질에 문제가 많다. 즉 labeling까지 자동화하기에 무리가 있는 것 같다.

# 훈련
- pre define 된 모델을 사용하고 학습에 필요한 데이터를 적절히 넣어주면 될 것 같았는데 너무 naive 했다.
``` python
model_name = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
```
- 모델 선택도 잘못되었다. Classfication model을 왜 썼을까?!
    - chat 선생님 가이드를 맹신하지 말자
    - 코드를 한줄 한줄 점검해야 하는데 그러지 못했다.
    - 아무리 쉬워 보여도 직접 하면 어렵고 모든 머신러닝 코드는 매우 짧다.
    - 한줄에 담긴 내용을 모두 소화해야 한다.

# 계획
- 설교 요약본이 digital 자료로 있으니 구해보자.
- 이 자료와 STT 결과를 비교해서 교정 작업을 해보자.
- 참고로 haspell과 pykospacing 등을 써봤지만 모두 효과가 없었다.
- 원문을 단락으로는 나눌 수 있기 때문에 (첫 번째, 두 번째.. ) 이를 최대한 사용하자.
- 기본 아이디어
    - 원문 텍스트와 요약 텍스트를 각각 단락 단위로 나누고, 유사도를 계산하여 가장 적합한 매칭을 찾는다.
    - 요약 텍스트의 높은 품질을 바탕으로 원문 텍스트를 교정
- 약점 및 해결
    - 말 그대로 요약 텍스트이기 때문에 원문의 용어를 그대로 사용하지는 않는다.
    - 따라서 문장을 벡터와해서 유사도를 찾는다면 결과가 나오지 않을것이다.
    - 예상할 수 있는 계획으로는 의미론적 유사도를 찾는 것이다.
- 세부 구현
    - 요약 텍스트를 원문과 유사한 표현으로 확장
        - 요약 텍스트를 확장하거나 원문과의 관계를 강화하기 위해 생성 모델을 활용합니다.
        - 예: T5, GPT 모델을 사용하여 요약 텍스트를 "원문과 유사한 표현으로 변환".                    
        > T5는 Text-To-Text Transfer Transformer의 약자로, Google이 개발한 자연어 처리(NLP) 모델입니다. T5는 NLP 작업(질문-답변, 요약, 번역, 분류 등)을 모두 텍스트 입력 → 텍스트 출력의 통합된 형식으로 처리하는 것이 특징입니다.
    - 요약 텍스트를 임베딩 벡터로 변환 후 원문과 비교
        - 요약 텍스트와 원문 텍스트를 벡터로 변환해 의미적 유사도를 기반으로 비교합니다.
    - 요약 텍스트에서 키워드 추출
        - 요약 텍스트의 핵심 키워드를 추출하고 이를 원문 텍스트 내에서 탐색.    